<a href="https://colab.research.google.com/github/dylan0stewart/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Dylan_Stewart_assignment_regression_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
df.head()

In [18]:
''' Gotta engineer a couple features, so I'm going to create columns for these:
      -Does the apartment accept both cats and dogs?
      and
      -The ratio of bedrooms to bathrooms
'''

#Let's engineer some features!

# Bedrooms per Bathrooms
df['Beds_per_Bath'] = df['bedrooms'] / df['bathrooms']

# Drop NaNs
df.dropna

# Do they allow both Cats and Dogs
# First, combine the two columns
df['cats_and_dogs'] = df['cats_allowed'] + df['dogs_allowed']
# Now, change 1's to 0's, since I only want the 2's(2 means they allow both)
df['cats_and_dogs'] = df['cats_and_dogs'].replace(1, 0)

# Check to make sure the features are added and accurate
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,Beds_per_Bath,cats_and_dogs
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,0


In [19]:
'''
Next, I need to split the data into training and test!
April and May = Training data
June = Test data
'''

# Check the date format to make sure its in datetime
df['created'].dtypes

# Time to split the frame up!
test = df[pd.to_datetime(df['created']).dt.strftime('%B') == 'June']

# Train Data is a lttle harder, maybe there is an easier way but this is all I could think of
may = df[pd.to_datetime(df['created']).dt.strftime('%B') == 'May']
april = df[pd.to_datetime(df['created']).dt.strftime('%B') == 'April']
train = pd.concat([may, april])

# Success with both! Let's just doublecheck that it did what I wanted
test.head()
train.tail()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,Beds_per_Bath,cats_and_dogs
49346,1.0,1,2016-04-22 15:44:11,24hr Doorman Luxury building in the heart of t...,East 10th Street,40.7296,-73.9869,4500,166 2nd avenue,medium,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1.0,2
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2
49351,1.0,2,2016-04-12 02:48:07,Look no further!!! This giant 2 bedroom apart...,Park Terrace East,40.8699,-73.9172,2200,30 Park Terrace East,low,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,0


In [0]:
''' 
Now to move onto the real stuff: building a linear regression model with
 at least 2 features!
For features, I'm going to use:
-cats_and_dogs, and beds_per_bath-
this is because if I was actually looking, those would be 
what I would look for
'''
# Lets put my features and target into a scatter!

# Import plotly
import plotly.express as px

# Plot it out
px.scatter_3d(
    train,
    x='Beds_per_Bath', 
    y='cats_and_dogs', 
    z='price', 
    text='created', 
    title='Apartments in NYC for rent'
)


### Didnt do the scatter because i dont neeeeed it, and im scared
# Last time i tried to plot this scatter it screwed up my colab entirely and
#   wouldnt let me run anything for like an hour

In [22]:
features = ['bathrooms', 
            'cats_and_dogs']
print(f'Linear Regression, dependent on: {features}')


# create my X_train/test and y_train/test
X_train = train[features]
X_test = test[features]

target = 'price'
y_train = train[target]
y_test = test[target]

Linear Regression, dependent on: ['bathrooms', 'cats_and_dogs']


In [23]:
# Fit the model

# import linreg and mae
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# instantiate linreg
model = LinearRegression()

# Now actually fit it
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)

# My MAE
mae

883.8429136493424

In [24]:
# My model's intercept and coefficient
model.intercept_, model.coef_

(439.48674034168744, array([2547.49025192,   78.21501407]))

In [0]:
coef_mean = model.coef_.mean()

In [32]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# MAE, RMSE, R^2, MSE for Training data
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1645699.9709129566
Root Mean Squared Error: 1282.8483818881157
Mean Absolute Error: 883.8429136493424
R^2: 0.46998872299118033


In [0]:
model.fit(X_test, y_test)
y_pred_test = model.predict(X_test)

In [36]:
# MAE, RMSE, R^2, MSE for Testing data
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_test)
r2 = r2_score(y_test, y_pred_test)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1607909.4762386335
Root Mean Squared Error: 1268.0337046934649
Mean Absolute Error: 890.2296431145531
R^2: 0.4826581488132502
